# Replication Notebooks
[The Impact of High School Financial Education: Evidence from a Large-Scale Evaluation in Brazil]( https://doi.org/10.1257/app.20150149.) 

By [Charlie Zhang](mailto:charlie.zhang@georgetown.edu)

In [1]:
import os
os.chdir("../")

import pandas as pd
import numpy as np
import statsmodels as sm
import statsmodels.formula.api as smf
import scipy

import warnings
warnings.filterwarnings("ignore")

In [2]:
panel_final = os.getcwd() + "/Data/school_intervention_panel_final.dta"

In [3]:
pf = pd.read_stata(panel_final)
pf.head(5)

,id_geral,cd_escola,nm_uf_bl,matriculas,docentes,abandonona1sriemdio,aprovaona1sriemdio,treatment,pair_all,treatment_workshop,...,dumm_rp_18p_fup,dumm_rp_19p_fup,dumm_rp_21p_fup,dumm_rp_23p_fup,dumm_formal_saving_fup,dumm_rp_33p_fup,dumm_rp_34p_fup,dumm_rp_36p_fup,dumm_rp_37p_fup,dumm_rp_41p_fup
0,1,17002648.0,TOCANTINS,273.0,34.0,6.8,76.7,yes,17017.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,17002648.0,TOCANTINS,273.0,34.0,6.8,76.7,yes,17017.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,17018390.0,TOCANTINS,641.0,29.0,4.2,90.7,no,17008.0,NaN,...,1.0,1.0,NaN,NaN,1.0,1.0,0.0,0.0,0.0,NaN
3,10,17018390.0,TOCANTINS,641.0,29.0,4.2,90.7,no,17008.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100,33002614.0,RIO DE JANEIRO,199.0,35.0,13.3,80.0,yes,33031.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,NaN


In [4]:
# Line 41-43
pf = (pf[(pf["round"] == "yes")& (pf["treatment"].isna() == False)]
      .sort_values(by=["cd_escola", "id_geral"])
      .reset_index()
      .drop("index", axis=1))
pf.head(5)

,id_geral,cd_escola,nm_uf_bl,matriculas,docentes,abandonona1sriemdio,aprovaona1sriemdio,treatment,pair_all,treatment_workshop,...,dumm_rp_18p_fup,dumm_rp_19p_fup,dumm_rp_21p_fup,dumm_rp_23p_fup,dumm_formal_saving_fup,dumm_rp_33p_fup,dumm_rp_34p_fup,dumm_rp_36p_fup,dumm_rp_37p_fup,dumm_rp_41p_fup
0,10166,17000386.0,TOCANTINS,414.0,42.0,9.6,65.2,yes,17006.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10439,17000386.0,TOCANTINS,414.0,42.0,9.6,65.2,yes,17006.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10448,17000386.0,TOCANTINS,414.0,42.0,9.6,65.2,yes,17006.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10568,17000386.0,TOCANTINS,414.0,42.0,9.6,65.2,yes,17006.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13397,17000386.0,TOCANTINS,414.0,42.0,9.6,65.2,yes,17006.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Table 1

In [5]:
test = [
    "female", "dumm_rp_08_bl", "dumm_rp_09_bl", "dumm_rp_24_bl",
    "dumm_rp_14_bl", "dumm_rp_23_bl", "vl_proficiencia_bl"
]
aluno = [
    "dumm_rp_49_bl", "dumm_rp_50_bl", "dumm_rp_65A_bl", "poupar_final2_bl",
    "dumm_rp_64A_bl", "dumm_negotiates_bl", "autonomia_final2_bl"
]
school = [
    "matriculas", "docentes", "abandonona1sriemdio", "aprovaona1sriemdio"
]
xvars = test + aluno

In [8]:
def ttest(stats, alpha=0.05):

    mean_1, mean_2 = stats[(var, "mean")]
    std_1, std_2 = stats[(var, "std")]
    n_1, n_2 = stats[(var, "count")]
    se_1, se_2 = std_1 / np.sqrt(n_1), std_2 / np.sqrt(n_2)
    sed = np.sqrt(se_1**2.0 + se_1**2.0)
    t_stat = (mean_1 - mean_2) / sed
    df = n_1 + n_2 - 2
    cv = scipy.stats.t.ppf(1.0 - alpha, df)
    p = (1.0 - scipy.stats.t.cdf(abs(t_stat), df)) * 2.0
    return p

In [9]:
t1p1 = pd.DataFrame()
for var in school:
    temp = ["treatment", "cd_escola"]
    temp.append(var)
    pf_temp = (pf[temp].drop_duplicates()[["treatment", var]])
    stats = pf_temp.groupby("treatment").agg({var: ["mean", "std", "count"]})
    stats[(var, "t-test")] = ttest(stats)
    t1p1 = pd.concat([t1p1, stats], axis=1).round(4)

display(t1p1)

matriculas                         docentes                         \
                mean       std count  t-test     mean      std count  t-test   
treatment                                                                      
no          642.8363  461.6997   452  0.2156  37.4949  24.0912   392  0.6504   
yes         680.8932  514.6180   440  0.2156  38.2751  25.7419   378  0.6504   

          abandonona1sriemdio                        aprovaona1sriemdio  \
                         mean      std count  t-test               mean   
treatment                                                                 
no                    11.0351  11.2233   447  0.3488            68.0510   
yes                   11.7389  11.8174   435  0.3488            67.7446   

                                  
               std count  t-test  
treatment                         
no         16.0285   447  0.7751  
yes        15.9359   435  0.7751

In [10]:
t1p2 = pd.DataFrame()
for var in xvars:
    temp = ["treatment", "id_geral"]
    temp.append(var)
    pf_temp = (pf[temp].drop_duplicates()[["treatment", var]])
    stats = pf_temp.groupby("treatment").agg({var: ["mean", "std", "count"]})
    stats[(var, "t-test")] = ttest(stats)
    t1p2 = pd.concat([t1p2, stats], axis=1).round(3)

display(t1p2.T)

treatment                             no           yes
female              mean        0.549000      0.563000
                    std         0.498000      0.496000
                    count   11954.000000  11529.000000
                    t-test      0.034000      0.034000
dumm_rp_08_bl       mean        0.446000      0.435000
                    std         0.497000      0.496000
                    count   11774.000000  11350.000000
                    t-test      0.111000      0.111000
dumm_rp_09_bl       mean        0.414000      0.405000
                    std         0.493000      0.491000
                    count   11720.000000  11283.000000
                    t-test      0.133000      0.133000
dumm_rp_24_bl       mean        0.297000      0.315000
                    std         0.457000      0.465000
                    count   11741.000000  11335.000000
                    t-test      0.002000      0.002000
dumm_rp_14_bl       mean        0.316000      0.333000
                    std         0.465000      0.471000
                    count   11878.000000  11435.000000
                    t-test      0.004000      0.004000
dumm_rp_23_bl       mean        0.515000      0.529000
                    std         0.500000      0.499000
                    count   11783.000000  11363.000000
                    t-test      0.037000      0.037000
vl_proficiencia_bl  mean       49.868000     50.153000
                    std        15.163000     14.830000
                    count   12173.000000  11751.000000
                    t-test      0.142000      0.142000
dumm_rp_49_bl       mean        0.352000      0.351000
                    std         0.478000      0.477000
                    count   12769.000000  12229.000000
                    t-test      0.871000      0.871000
dumm_rp_50_bl       mean        0.661000      0.672000
                    std         0.473000      0.469000
                    count   12778.000000  12234.000000
                    t-test      0.073000      0.073000
dumm_rp_65A_bl      mean        0.153000      0.161000
                    std         0.360000      0.367000
                    count   12533.000000  12003.000000
                    t-test      0.092000      0.092000
poupar_final2_bl    mean       48.360001     48.175999
                    std        18.911000     18.557000
                    count   11985.000000  11479.000000
                    t-test      0.453000      0.453000
dumm_rp_64A_bl      mean        0.103000      0.105000
                    std         0.304000      0.307000
                    count   12657.000000  12109.000000
                    t-test      0.566000      0.566000
dumm_negotiates_bl  mean        0.753000      0.758000
                    std         0.431000      0.429000
                    count   12525.000000  12030.000000
                    t-test      0.421000      0.421000
autonomia_final2_bl mean       49.077000     48.981998
                    std        19.806000     19.582000
                    count   12005.000000  11445.000000
                    t-test      0.711000      0.711000

## Table 2